In [ ]:
import numpy as np
import teradata
import pandas as pd
from pandas import Series,DataFrame

import math

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

from sklearn import metrics
import statsmodels.api as sm
from sklearn import preprocessing
import glob
import datetime
import csv

In [ ]:
path =r'/Users/yuanyao/Documents/underwriting/underwriting/' # use your path
allFiles = glob.glob(path + "/*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,sep=",",dtype=object,index_col=None, header=None)
    list_.append(df)
frame = pd.concat(list_)

In [ ]:
hold_path =r'/Users/yuanyao/Documents/underwriting/underwriting_hold/' # use your path
allFiles = glob.glob(hold_path + "/*.txt")
hold_frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,sep=",",dtype=object,index_col=None, header=None)
    list_.append(df)
hold_frame = pd.concat(list_)

In [ ]:
sql_query=""" select * from pp_oap_yaoy_t.uw_recent_dt ;"""
uw_recent_dt=pd.read_sql(sql_query,simba)

In [ ]:
# the am UW send
uw_score=uw_no_am.loc[uw_no_am['score']<740]
uw_am=uw_score.loc[~uw_score['cust_id'].isin(hold_frame[1])]
uw_am['GRP']='Organic Underwriting'
uw_am_send=uw_am[['cust_id','busn_name','GRP','AM_NAME','AM_SEGMENT','reasons']]

In [ ]:
# the AM PR send
pr_am_score=pr_am.loc[pr_am['score']<790]
pr_am_score.shape
pr_am_dedup=pr_am_score.loc[~pr_am_score['cust_id'].isin(hold_frame[1])]
pr_am_dedup=pr_am_dedup.loc[~pr_am_dedup['cust_id'].isin(frame[1])]
pr_am_dedup['GRP']='Periodic Review'
pr_am_sent=pr_am_dedup[['cust_id','busn_name','GRP','AM_NAME','AM_SEGMENT','reasons']]
pr_am_sent=pr_am_sent.drop_duplicates(subset=['cust_id'], keep="last")

In [ ]:
# the am PR send more
pr_am_dedup_more=pr_am_more.loc[~pr_am_more['cust_id'].isin(hold_frame[1])]
pr_am_dedup_more=pr_am_dedup_more.loc[~pr_am_dedup_more['cust_id'].isin(frame[1])]
pr_am_dedup_more=pr_am_dedup_more.loc[pr_am_dedup_more['score']<850]
pr_am_dedup_more['GRP']='Periodic Review'
pr_am_sent_more=pr_am_dedup_more[['cust_id','busn_name','GRP','AM_NAME','AM_SEGMENT','reasons']]
pr_am_sent_more=pr_am_sent_more.drop_duplicates(subset=['cust_id'], keep="last")

In [ ]:
# prepare the deduped archive dataset
am_send=pd.concat([uw_am_send, pr_am_sent,pr_am_sent_more], ignore_index=True)
am_send_dedup=am_send.drop_duplicates(subset=['cust_id'], keep="last")

In [ ]:
am_send_dedup

In [ ]:
# archive the am send including UW and PR
send_date=datetime.date.today()+datetime.timedelta(14)
am_send_dedup['send_dt']=send_date.strftime('%Y%m%d')
df_obj = am_send_dedup.select_dtypes(['object'])
am_send_dedup[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
am_send_dedup=am_send_dedup.loc[~am_send_dedup['cust_id'].isin(hold_frame[1])]
am_send_dedup=am_send_dedup.loc[~am_send_dedup['cust_id'].isin(frame[1])]

In [ ]:
# send out as email
am_send_dedup

In [ ]:
# put into txt file
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_archive='zoot_all_send'+ today_path +'_hold.txt'
am_send_dedup.to_csv(path_archive+filename_archive, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [ ]:
# prepare the am send file
am_send_ow=am_send_dedup[am_send_dedup['GRP']=='Organic Underwriting']
am_send_ow['upd_Referral_Codes']='ORGUWD'
am_send_ow['Account_Number']=am_send_ow['cust_id']
send_date=datetime.date.today()+datetime.timedelta(14)
am_send_ow['Referral_Date']=send_date.strftime('%Y%m%d')
am_send_ow['_comment']=am_send_ow['reasons']
am_send_ow['_routing_reason_codes']=np.nan
am_send_ow['Priority_Score']=750
am_send_ow['Detections_Exposure']=0
am_send_ow['CAP_Value']=0
am_send_ow['EBay_AVERAGE_Pricing']=0
am_send_ow['_industry']=np.nan
am_send_ow['_merchant_type_long']="D2S"
am_send_ow['Thirty_Day_Rolling_TPV']=0
am_send_ow['Shadow_Monit_Flag']=0
am_send_ow['Account_Managed_Flag']=0
df_obj = am_send_ow.select_dtypes(['object'])
am_send_ow[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
am_send_csv=am_send_ow[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_ow='zoot_ouw_referral_NA_'+ today_path +'_hold.txt'
filename_ow
am_send_csv.to_csv(hold_path+filename_ow, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [ ]:
# prepare the file for am PR
am_send_pr=am_send_dedup[am_send_dedup['GRP']=='Periodic Review']
am_send_pr['upd_Referral_Codes']='PERIODICREV'
am_send_pr['Account_Number']=am_send_pr['cust_id']
send_date=datetime.date.today()+datetime.timedelta(14)
am_send_pr['Referral_Date']=send_date.strftime('%Y%m%d')
am_send_pr['_comment']=am_send_pr['reasons']
am_send_pr['_routing_reason_codes']=np.nan
am_send_pr['Priority_Score']=750
am_send_pr['Detections_Exposure']=0
am_send_pr['CAP_Value']=0
am_send_pr['EBay_AVERAGE_Pricing']=0
am_send_pr['_industry']=np.nan
am_send_pr['_merchant_type_long']="D2S"
am_send_pr['Thirty_Day_Rolling_TPV']=0
am_send_pr['Shadow_Monit_Flag']=0
am_send_pr['Account_Managed_Flag']=0
df_obj = am_send_pr.select_dtypes(['object'])
am_send_pr[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
am_send_pr_csv=am_send_pr[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_pr='zoot_PR_referral_NA_'+ today_path +'_hold.txt'
am_send_pr_csv.to_csv(hold_path+filename_pr, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [ ]:
ow_nonam=pd.read_sql(sql_uw_nonam,simba)

In [ ]:
uw_noam=ow_nonam.loc[~ow_nonam['cust_id'].isin(frame[1])]

In [ ]:
uw_noam_score=uw_noam.loc[uw_noam['score']<690]

In [ ]:
uw_noam_dedup=uw_noam_score.loc[~uw_noam_score['cust_id'].isin(frame[1])]
uw_noam_dedup['GRP']='Organic Underwriting'
uw_noam_sent=uw_noam_dedup[['cust_id','GRP','AM_NAME','AM_SEGMENT','reasons','most_recent_rvw_dt','Auto_Tier']]
uw_noam_sent=uw_noam_sent.drop_duplicates(subset=['cust_id'], keep="last")

In [ ]:
uw_noam_sent

In [ ]:
# start the non-am pr
sql_pr_nonam="""SEL 
base.cust_id, base.reason as reasons
, 'Periodic Review' as grp
, base.am_name
, base.am_segment
, date as cutoff_dt
, base.score
, ' ' as Auto_Tier
, base.most_recent_rvw_dt
, 'PR' as seg
, date as send_dt
FROM pp_oap_yaoy_t.pr_routing base
WHERE (base.most_recent_rvw_dt <= date -180 or base.most_recent_rvw_dt is null)
AND (base.am_name in ('BSS','?', 'Channel Partner', 'Regional AM','LE') OR base.am_name IS NULL)
And base.most_recent_rvw_dt<= current_date-210
ORDER BY base.score asc
"""

In [ ]:
pr_nonam=pd.read_sql(sql_pr_nonam,simba)
pr_noam_dedup=pr_nonam.loc[~pr_nonam['cust_id'].isin(frame[1])]
pr_noam_score=pr_noam_dedup.loc[pr_noam_dedup['score']<720]
pr_noam_dedup=pr_noam_score.loc[~pr_noam_score['cust_id'].isin(frame[1])]
pr_noam_dedup['GRP']='Periodic Review'
pr_noam_sent=pr_noam_dedup[['cust_id','GRP','AM_NAME','AM_SEGMENT','reasons','most_recent_rvw_dt','Auto_Tier']]
pr_noam_sent=pr_noam_sent.drop_duplicates(subset=['cust_id'], keep="last")
pr_noam_sent

In [ ]:
pr_nonam_more=pd.read_sql(sql_pr_nonam_more,simba)
pr_noam_more_dedup=pr_nonam_more.loc[~pr_nonam_more['cust_id'].isin(frame[1])]
pr_noam_more_score=pr_noam_more_dedup.loc[pr_noam_more_dedup['score']<720]
pr_noam_more_dedup=pr_noam_more_score.loc[~pr_noam_more_score['cust_id'].isin(frame[1])]
pr_noam_more_dedup['GRP']='Periodic Review'
pr_noam_more_sent=pr_noam_more_dedup[['cust_id','GRP','AM_NAME','AM_SEGMENT','reasons','most_recent_rvw_dt','Auto_Tier']]
pr_noam_more_sent=pr_noam_more_sent.drop_duplicates(subset=['cust_id'], keep="last")

In [ ]:
pr_noam_more_sent

In [ ]:
# prepare the deduped archive dataset
nonam_send=pd.concat([uw_noam_sent, pr_noam_sent,pr_noam_more_sent], ignore_index=True)
noam_send_dedup=nonam_send.drop_duplicates(subset=['cust_id'], keep="last")
noam_send_dedup=noam_send_dedup.loc[~noam_send_dedup['cust_id'].isin(frame[1])]

In [ ]:
noam_send_dedup

In [ ]:
# archive the noam send including UW and PR
send_date=datetime.date.today()+datetime.timedelta(14)
noam_send_dedup['send_dt']=send_date.strftime('%Y%m%d')
df_obj = noam_send_dedup.select_dtypes(['object'])
noam_send_dedup[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_archive='zoot_all_send'+ today_path +'.txt'
noam_send_dedup.to_csv(path_archive+filename_archive, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [ ]:
# prepare the UW nonam send
noam_send_uw_final=noam_send_dedup[noam_send_dedup['GRP']=='Organic Underwriting']
noam_send_uw_final['upd_Referral_Codes']='ORGUWD'
noam_send_uw_final['Account_Number']=noam_send_uw_final['cust_id']
noam_send_uw_final['Referral_Date']=pd.Timestamp('today').strftime('%Y%m%d')
noam_send_uw_final['_comment']=noam_send_uw_final['reasons']
noam_send_uw_final['_routing_reason_codes']=np.nan
noam_send_uw_final['Priority_Score']=730
noam_send_uw_final['Detections_Exposure']=0
noam_send_uw_final['CAP_Value']=0
noam_send_uw_final['EBay_AVERAGE_Pricing']=0
noam_send_uw_final['_industry']=np.nan
noam_send_uw_final['_merchant_type_long']="D2S"
noam_send_uw_final['Thirty_Day_Rolling_TPV']=0
noam_send_uw_final['Shadow_Monit_Flag']=0
noam_send_uw_final['Account_Managed_Flag']=0
df_obj = noam_send_uw_final.select_dtypes(['object'])
noam_send_uw_final[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
noam_send_uw_csv=noam_send_uw_final[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_ow='zoot_ouw_referral_NA_'+ today_path +'.txt'
noam_send_uw_csv.to_csv(path+filename_ow, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [ ]:
noam_send_uw_csv

In [ ]:
# prepare the pr nonam send
noam_send_pr_final=noam_send_dedup[noam_send_dedup['GRP']=='Periodic Review']
noam_send_pr_final['upd_Referral_Codes']='PERIODICREV'
noam_send_pr_final['Account_Number']=noam_send_pr_final['cust_id']
noam_send_pr_final['Referral_Date']=pd.Timestamp('today').strftime('%Y%m%d')
noam_send_pr_final['_comment']=noam_send_pr_final['reasons']
noam_send_pr_final['_routing_reason_codes']=np.nan
noam_send_pr_final['Priority_Score']=720
noam_send_pr_final['Detections_Exposure']=0
noam_send_pr_final['CAP_Value']=0
noam_send_pr_final['EBay_AVERAGE_Pricing']=0
noam_send_pr_final['_industry']=np.nan
noam_send_pr_final['_merchant_type_long']="D2S"
noam_send_pr_final['Thirty_Day_Rolling_TPV']=0
noam_send_pr_final['Shadow_Monit_Flag']=0
noam_send_pr_final['Account_Managed_Flag']=0
df_obj = noam_send_pr_final.select_dtypes(['object'])
noam_send_pr_final[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
noam_send_pr_csv=noam_send_pr_final[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_pr='zoot_PR_referral_NA_'+ today_path +'.txt'
noam_send_pr_csv.to_csv(path+filename_pr, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')

In [ ]:
noam_send_pr_csv

In [ ]:
# prepare the pr nonam send change with combine
noam_send_pr_final=noam_send_dedup[noam_send_dedup['GRP']=='Periodic Review']
noam_send_pr_final['upd_Referral_Codes']='PERIODICREV'
noam_send_pr_final['Account_Number']=noam_send_pr_final['cust_id']
noam_send_pr_final['Referral_Date']=pd.Timestamp('today').strftime('%Y%m%d')
noam_send_pr_final['_comment']=noam_send_pr_final['reasons']
noam_send_pr_final['_routing_reason_codes']=np.nan
noam_send_pr_final['Priority_Score']=750
noam_send_pr_final['Detections_Exposure']=0
noam_send_pr_final['CAP_Value']=0
noam_send_pr_final['EBay_AVERAGE_Pricing']=0
noam_send_pr_final['_industry']=np.nan
noam_send_pr_final['_merchant_type_long']="D2S"
noam_send_pr_final['Thirty_Day_Rolling_TPV']=0
noam_send_pr_final['Shadow_Monit_Flag']=0
noam_send_pr_final['Account_Managed_Flag']=0
noam_send_pr_csv=noam_send_pr_final[['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']]
# attach hold file
cols=['upd_Referral_Codes','Account_Number','Referral_Date','_comment','_routing_reason_codes','Priority_Score','Detections_Exposure','CAP_Value','EBay_AVERAGE_Pricing','_industry','_merchant_type_long'
            ,'Thirty_Day_Rolling_TPV','Shadow_Monit_Flag','Account_Managed_Flag']
pre_pr_hold = pd.read_csv(hold_path+'zoot_PR_referral_NA_20170807_hold.txt', names=cols,header=None)
pr_send_all=pd.concat([pre_pr_hold,noam_send_pr_csv], ignore_index=True)
# strip column
df_obj = pr_send_all.select_dtypes(['object'])
pr_send_all[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

today_path=pd.Timestamp('today').strftime('%Y%m%d')
filename_pr='zoot_PR_referral_NA_'+ today_path +'.txt'
# craete send file
pr_send_all.to_csv(path+filename_pr, header=None, index=None, sep=',',quoting=csv.QUOTE_NONNUMERIC,encoding='ascii')